In [1]:
#filename = "ml.py"
#exec(compile(open(filename, "rb").read(), filename, 'exec'))

In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from sklearn.model_selection import train_test_split
from nltk.sentiment.vader import SentimentIntensityAnalyzer
import nltk
import dateutil.parser as parser
from sklearn.linear_model import LogisticRegression

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Input

In [3]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from sklearn.model_selection import train_test_split
from nltk.sentiment.vader import SentimentIntensityAnalyzer
import nltk

nltk.download('vader_lexicon')

# read the dataset into pandas dataframe
# df = pd.read_csv('./../downsampled_data', delim_whitespace=False).dropna()
df = pd.read_csv('./../downsampled_data.csv').dropna()

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /Users/trevorcarpenter/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [4]:
df.columns

Index(['Unnamed: 0', 'video_id', 'title', 'publishedAt', 'channelId',
       'channelTitle', 'categoryId', 'duration', 'definition', 'trended_later',
       'time_retrieved', 'tags', 'thumbnail_link', 'ratings_disabled',
       'description', 'Channel_viewCount', 'Channel_subscriberCount',
       'Channel_hiddenSubscriberCount', 'Channel_videoCount', 'Channel_title',
       'Channel_description', 'Channel_publishedAt', 'Channel_country',
       'view_count_update_11_19_14', 'likes_update_11_19_14',
       'dislikes_update_11_19_14', 'comment_count_update_11_19_14',
       '11_19_14_update_timestamp', 'trending?', 'engagement_rate', 'INTL',
       'UNK', 'USA'],
      dtype='object')

In [5]:
df['categoryId'] = df['categoryId'].astype(np.float)

categories = pd.DataFrame(np.zeros((len(df), len(df['categoryId'].unique()))), columns = df['categoryId'].unique())
index = 0
for id in df['categoryId']:
    categories[id][index] = 1
    index += 1

In [6]:
# display the first five rows
retrieval_time = df['time_retrieved']
publish_time = df['publishedAt']
channel_publish_time = df['Channel_publishedAt']
retrieval_time_11_19_14 = df['11_19_14_update_timestamp']
columns_to_drop = ['Unnamed: 0', 'definition', 'categoryId', 'publishedAt', 'time_retrieved', 'Channel_title', '11_19_14_update_timestamp', 'Channel_publishedAt', 'video_id', 'channelId', 'thumbnail_link', 'Channel_country']
df = df.drop(columns_to_drop, axis = 1)

In [7]:
categories.columns

Float64Index([24.0, 25.0, 26.0, 17.0, 22.0, 20.0,  1.0, 10.0, 23.0, 28.0,  2.0,
              27.0, 19.0, 15.0, 30.0, 29.0],
             dtype='float64')

In [8]:
index = 0
for column in categories.columns:
    df.insert(df.shape[1], "Category_"+str(column), categories[column])
    index += 1

In [9]:
## time updates
import dateutil.parser as parser
age = []
age_update = []
channel_age = []
for i in df.index:
    channel_publish_time[i] = channel_publish_time[i].replace("\"", "")
    age.append(parser.isoparse(retrieval_time[i]) - parser.isoparse(publish_time[i]))
    age_update.append(parser.isoparse(retrieval_time_11_19_14[i]) - parser.isoparse(publish_time[i]))
    channel_age.append(parser.isoparse(channel_publish_time[i]) - parser.isoparse(publish_time[i]))
    if df['ratings_disabled'][i] == 'True':
        df['ratings_disabled'][i] = True
    elif df['ratings_disabled'][i] == 'False':
        df['ratings_disabled'][i] = False
        
    if df['Channel_hiddenSubscriberCount'][i] == 'True':
        df['Channel_hiddenSubscriberCount'][i] = True
    elif df['Channel_hiddenSubscriberCount'][i] == 'False':
        df['Channel_hiddenSubscriberCount'][i] = False
        
        
    if df['trended_later'][i] == 'True':
        df['trended_later'][i] = True
    elif df['trended_later'][i] == 'False':
        df['trended_later'][i] = False

/Applications/anaconda2/envs/newEnv/lib/python3.7/site-packages/ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/Applications/anaconda2/envs/newEnv/lib/python3.7/site-packages/ipykernel_launcher.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/Applications/anaconda2/envs/newEnv/lib/python3.7/site-packages/ipykernel_launcher.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/Applications/anaconda2/envs/newEn

In [10]:
## sentiment values

titles = df['title']
channel_title = df['channelTitle']
description = df['description']
channel_description = df['Channel_description']

title_sentiment_vals = []
channel_title_sentiment_vals = []
description_sentiment_vals = []
channel_description_sentiment_vals = []

sid = SentimentIntensityAnalyzer()
for sentence in titles:
    ss = sid.polarity_scores(str(sentence))
    title_sentiment_vals.append(ss['pos']-ss['neg'])
    
for sentence in channel_title:
    ss = sid.polarity_scores(str(sentence))
    channel_title_sentiment_vals.append(ss['pos']-ss['neg'])
    
for sentence in description:
    ss = sid.polarity_scores(str(sentence))
    description_sentiment_vals.append(ss['pos']-ss['neg'])
    
for sentence in channel_description:
    ss = sid.polarity_scores(str(sentence))
    channel_description_sentiment_vals.append(ss['pos']-ss['neg'])

In [11]:
df

,title,channelTitle,duration,trended_later,tags,ratings_disabled,description,Channel_viewCount,Channel_subscriberCount,Channel_hiddenSubscriberCount,...,Category_1.0,Category_10.0,Category_23.0,Category_28.0,Category_2.0,Category_27.0,Category_19.0,Category_15.0,Category_30.0,Category_29.0
0,Biden Victory Cold Open - SNL,Saturday Night Live,-0.0707230866960673,False,36.0,False,Joe Biden (Jim Carrey) and Donald Trump (Alec ...,0.819940,0.504608,False,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,How President Trump and the White House reacte...,CNBC Television,-0.0856798827419832,False,28.0,False,CNBC's Kayla Tausche joins The News with Shepa...,-0.274970,-0.350255,False,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,Watch Live: President-Elect Joe Biden Addresse...,ABC News,0.09488158288091483,False,13.0,False,#Election2020 #Election #JoeBiden #PresidentEl...,0.650191,0.461214,False,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,I GOT PLASTIC SURGERY TO SEE HOW MY FRIENDS RE...,Alexa Rivera,-0.07104706061763949,False,0.0,False,"This was definitely such a fun prank, their re...",-0.258217,0.002973,False,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,FC Barcelona 5 - 2 Real Betis - HIGHLIGHTS & G...,beIN SPORTS USA,-0.07288291283988188,False,12.0,False,Lionel Messi entró al segundo tiempo y resucit...,-0.330711,-0.407708,False,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
875,"Eying WH, Bullish Biden Campaign Tells Trump T...",MSNBC,-0.09237534378780837,False,32.0,False,As experts worry Trump will refuse to concede ...,0.202458,-0.117663,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
876,Belize: Joko gegen die Kokosnuss | Duell um di...,Joko & Klaas,-0.06953518231696929,False,26.0,False,Joko zerdeppert sich seinen Kopf an einer Koko...,-0.235549,-0.325954,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
877,Alex De Minaur vs Ugo Humbert | Antwerp 2020 F...,Tennis TV,-0.07223496499673751,False,25.0,False,Two young guns battle it out for the Antwerp t...,-0.300937,-0.396426,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
878,Pikmin 3 Deluxe: Top 5 Things to Know Before Y...,Nintendo,0.07792694765197046,False,43.0,False,With only a week to go before the Pikmin™ 3 De...,-0.063224,0.192171,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [12]:
from sklearn import preprocessing

df['title'] = title_sentiment_vals
df['channelTitle'] = channel_title_sentiment_vals
df['description'] = description_sentiment_vals
df['Channel_description'] = channel_description_sentiment_vals

le = preprocessing.LabelEncoder()
df['trending?'] = le.fit_transform(df['trending?'])
df['ratings_disabled'] = le.fit_transform(df['ratings_disabled'])
df['Channel_hiddenSubscriberCount'] = le.fit_transform(df['Channel_hiddenSubscriberCount'])
pd.set_option('display.max_columns', None)
df = df.dropna()

In [58]:
# dummy values for now - get these interactively later...
import tensorflow as tf

best_model = True
if best_model:
    num_hidden_layers = 3
    num_hidden_layer_nodes = [20, 40, 20]
    train_ratio = .7
    hidden_layer_activations = ['sigmoid', 'sigmoid', 'sigmoid', 'sigmoid']
    optimizer = 'adam'
    learning_rate = .005
    loss = 'mean_squared_error'
    metrics = [tf.keras.metrics.Accuracy(),tf.keras.metrics.Recall(),tf.keras.metrics.Precision()]
    metrics_names = ["accuracy","recall","precision"]
    epochs = 300
    batch_size = 200
else:
    # build a custom model
    num_hidden_layers = 3
    num_hidden_layer_nodes = [20, 10, 5]
    train_ratio = .7
    hidden_layer_activations = ['sigmoid', 'sigmoid', 'sigmoid', 'sigmoid']
    optimizer = 'adam'
    learning_rate = .005
    loss = 'mean_squared_error'
    metrics = [tf.keras.metrics.Accuracy(),tf.keras.metrics.Recall(),tf.keras.metrics.Precision()]
    metrics_names = ["accuracy","recall","precision"]
    epochs = 200
    batch_size = 200

In [59]:
# Split the data into training and testing set by 70:30
ratio = 0.7
train, test = train_test_split(df, train_size=ratio, random_state=42)
train.reset_index(inplace=True, drop=True)
test.reset_index(inplace=True, drop=True)

In [60]:
# separate data into x and y - just random y for now..
train_X = train.loc[:,train.columns != 'trending?']
train_Y = train['trending?']
test_X = test.loc[:,test.columns != 'trending?']
test_Y = test['trending?']

In [61]:
# build the logistic regression model - need clean data...
from sklearn.linear_model import LogisticRegression
LR_model = LogisticRegression(multi_class='ovr')
LR_model.fit(train_X, train_Y)

/Applications/anaconda2/envs/newEnv/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:764: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


LogisticRegression(multi_class='ovr')

In [62]:
# confusion matrix
from sklearn.metrics import confusion_matrix

pred_Y = LR_model.predict(test_X);
confusion_matrix(test_Y, pred_Y)

array([[134,   9],
       [ 22,  44]])

In [63]:
# evaluate the accuracy of the LR model
accuracy = LR_model.score(test_X, test_Y)
print(accuracy)

0.8516746411483254


In [72]:
# build the ANN model
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Input

ANN_model = keras.Sequential()

# add hidden layers
for i in range(num_hidden_layers):
    if i == 0:
        ANN_model.add(Dense(num_hidden_layer_nodes[i], input_dim = train_X.shape[1], activation=hidden_layer_activations[i]))
    ANN_model.add(Dense(num_hidden_layer_nodes[i], activation=hidden_layer_activations[i]))

# add output layers
ANN_model.add(Dense(1, activation=hidden_layer_activations[num_hidden_layers]))

ANN_model.compile(optimizer=optimizer, loss=loss, metrics=metrics)
ANN_model.summary()

Model: "sequential_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_35 (Dense)             (None, 20)                740       
_________________________________________________________________
dense_36 (Dense)             (None, 20)                420       
_________________________________________________________________
dense_37 (Dense)             (None, 40)                840       
_________________________________________________________________
dense_38 (Dense)             (None, 20)                820       
_________________________________________________________________
dense_39 (Dense)             (None, 1)                 21        
Total params: 2,841
Trainable params: 2,841
Non-trainable params: 0
_________________________________________________________________


In [73]:
# train the model
ANN_model.fit(train_X, train_Y, validation_data=(test_X, test_Y), epochs=epochs, batch_size=batch_size)

Train on 487 samples, validate on 209 samples
Epoch 1/300
487/487 [==============================] - 1s 2ms/sample - loss: 0.2727 - accuracy: 0.0000e+00 - recall_6: 1.0000 - precision_6: 0.2526 - val_loss: 0.2599 - val_accuracy: 0.0000e+00 - val_recall_6: 1.0000 - val_precision_6: 0.3158
Epoch 2/300
487/487 [==============================] - 0s 85us/sample - loss: 0.2596 - accuracy: 0.0000e+00 - recall_6: 1.0000 - precision_6: 0.2526 - val_loss: 0.2508 - val_accuracy: 0.0000e+00 - val_recall_6: 1.0000 - val_precision_6: 0.3158
Epoch 3/300
487/487 [==============================] - 0s 108us/sample - loss: 0.2482 - accuracy: 0.0000e+00 - recall_6: 0.3902 - precision_6: 0.2400 - val_loss: 0.2428 - val_accuracy: 0.0000e+00 - val_recall_6: 0.0000e+00 - val_precision_6: 0.0000e+00
Epoch 4/300
487/487 [==============================] - 0s 90us/sample - loss: 0.2378 - accuracy: 0.0000e+00 - recall_6: 0.0000e+00 - precision_6: 0.0000e+00 - val_loss: 0.2360 - val_accuracy: 0.0000e+00 - val_recal

Epoch 32/300
487/487 [==============================] - 0s 74us/sample - loss: 0.1887 - accuracy: 0.0000e+00 - recall_6: 0.0000e+00 - precision_6: 0.0000e+00 - val_loss: 0.2200 - val_accuracy: 0.0000e+00 - val_recall_6: 0.0000e+00 - val_precision_6: 0.0000e+00
Epoch 33/300
487/487 [==============================] - 0s 72us/sample - loss: 0.1887 - accuracy: 0.0000e+00 - recall_6: 0.0000e+00 - precision_6: 0.0000e+00 - val_loss: 0.2200 - val_accuracy: 0.0000e+00 - val_recall_6: 0.0000e+00 - val_precision_6: 0.0000e+00
Epoch 34/300
487/487 [==============================] - 0s 72us/sample - loss: 0.1886 - accuracy: 0.0000e+00 - recall_6: 0.0000e+00 - precision_6: 0.0000e+00 - val_loss: 0.2201 - val_accuracy: 0.0000e+00 - val_recall_6: 0.0000e+00 - val_precision_6: 0.0000e+00
Epoch 35/300
487/487 [==============================] - 0s 70us/sample - loss: 0.1886 - accuracy: 0.0000e+00 - recall_6: 0.0000e+00 - precision_6: 0.0000e+00 - val_loss: 0.2201 - val_accuracy: 0.0000e+00 - val_recall_

487/487 [==============================] - 0s 57us/sample - loss: 0.1876 - accuracy: 0.0000e+00 - recall_6: 0.0000e+00 - precision_6: 0.0000e+00 - val_loss: 0.2184 - val_accuracy: 0.0000e+00 - val_recall_6: 0.0000e+00 - val_precision_6: 0.0000e+00
Epoch 64/300
487/487 [==============================] - 0s 53us/sample - loss: 0.1875 - accuracy: 0.0000e+00 - recall_6: 0.0000e+00 - precision_6: 0.0000e+00 - val_loss: 0.2182 - val_accuracy: 0.0000e+00 - val_recall_6: 0.0000e+00 - val_precision_6: 0.0000e+00
Epoch 65/300
487/487 [==============================] - 0s 58us/sample - loss: 0.1873 - accuracy: 0.0000e+00 - recall_6: 0.0000e+00 - precision_6: 0.0000e+00 - val_loss: 0.2181 - val_accuracy: 0.0000e+00 - val_recall_6: 0.0000e+00 - val_precision_6: 0.0000e+00
Epoch 66/300
487/487 [==============================] - 0s 77us/sample - loss: 0.1872 - accuracy: 0.0000e+00 - recall_6: 0.0000e+00 - precision_6: 0.0000e+00 - val_loss: 0.2179 - val_accuracy: 0.0000e+00 - val_recall_6: 0.0000e+00

487/487 [==============================] - 0s 83us/sample - loss: 0.1771 - accuracy: 0.0000e+00 - recall_6: 0.0000e+00 - precision_6: 0.0000e+00 - val_loss: 0.2008 - val_accuracy: 0.0000e+00 - val_recall_6: 0.0000e+00 - val_precision_6: 0.0000e+00
Epoch 95/300
487/487 [==============================] - 0s 96us/sample - loss: 0.1762 - accuracy: 0.0000e+00 - recall_6: 0.0000e+00 - precision_6: 0.0000e+00 - val_loss: 0.1994 - val_accuracy: 0.0000e+00 - val_recall_6: 0.0000e+00 - val_precision_6: 0.0000e+00
Epoch 96/300
487/487 [==============================] - 0s 68us/sample - loss: 0.1752 - accuracy: 0.0000e+00 - recall_6: 0.0000e+00 - precision_6: 0.0000e+00 - val_loss: 0.1978 - val_accuracy: 0.0000e+00 - val_recall_6: 0.0000e+00 - val_precision_6: 0.0000e+00
Epoch 97/300
487/487 [==============================] - 0s 75us/sample - loss: 0.1742 - accuracy: 0.0000e+00 - recall_6: 0.0000e+00 - precision_6: 0.0000e+00 - val_loss: 0.1961 - val_accuracy: 0.0000e+00 - val_recall_6: 0.0000e+00

Epoch 126/300
487/487 [==============================] - 0s 87us/sample - loss: 0.1190 - accuracy: 0.0000e+00 - recall_6: 0.6098 - precision_6: 0.7653 - val_loss: 0.1165 - val_accuracy: 0.0000e+00 - val_recall_6: 0.7727 - val_precision_6: 0.8361
Epoch 127/300
487/487 [==============================] - 0s 86us/sample - loss: 0.1175 - accuracy: 0.0000e+00 - recall_6: 0.6260 - precision_6: 0.7624 - val_loss: 0.1147 - val_accuracy: 0.0000e+00 - val_recall_6: 0.7879 - val_precision_6: 0.8387
Epoch 128/300
487/487 [==============================] - 0s 64us/sample - loss: 0.1162 - accuracy: 0.0000e+00 - recall_6: 0.6423 - precision_6: 0.7596 - val_loss: 0.1132 - val_accuracy: 0.0000e+00 - val_recall_6: 0.7879 - val_precision_6: 0.8387
Epoch 129/300
487/487 [==============================] - 0s 79us/sample - loss: 0.1149 - accuracy: 0.0000e+00 - recall_6: 0.6504 - precision_6: 0.7619 - val_loss: 0.1117 - val_accuracy: 0.0000e+00 - val_recall_6: 0.7879 - val_precision_6: 0.8254
Epoch 130/300
48

487/487 [==============================] - 0s 73us/sample - loss: 0.1012 - accuracy: 0.0000e+00 - recall_6: 0.7398 - precision_6: 0.7339 - val_loss: 0.0986 - val_accuracy: 0.0000e+00 - val_recall_6: 0.8182 - val_precision_6: 0.7826
Epoch 160/300
487/487 [==============================] - 0s 64us/sample - loss: 0.1009 - accuracy: 0.0000e+00 - recall_6: 0.7398 - precision_6: 0.7339 - val_loss: 0.0990 - val_accuracy: 0.0000e+00 - val_recall_6: 0.8182 - val_precision_6: 0.7714
Epoch 161/300
487/487 [==============================] - 0s 73us/sample - loss: 0.1008 - accuracy: 0.0000e+00 - recall_6: 0.7398 - precision_6: 0.7280 - val_loss: 0.0991 - val_accuracy: 0.0000e+00 - val_recall_6: 0.8182 - val_precision_6: 0.7714
Epoch 162/300
487/487 [==============================] - 0s 70us/sample - loss: 0.1007 - accuracy: 0.0000e+00 - recall_6: 0.7398 - precision_6: 0.7339 - val_loss: 0.0987 - val_accuracy: 0.0000e+00 - val_recall_6: 0.8182 - val_precision_6: 0.7714
Epoch 163/300
487/487 [=======

487/487 [==============================] - 0s 64us/sample - loss: 0.0967 - accuracy: 0.0000e+00 - recall_6: 0.7561 - precision_6: 0.7440 - val_loss: 0.0984 - val_accuracy: 0.0000e+00 - val_recall_6: 0.8333 - val_precision_6: 0.7857
Epoch 193/300
487/487 [==============================] - 0s 60us/sample - loss: 0.0967 - accuracy: 0.0000e+00 - recall_6: 0.7805 - precision_6: 0.7385 - val_loss: 0.0986 - val_accuracy: 0.0000e+00 - val_recall_6: 0.8333 - val_precision_6: 0.7746
Epoch 194/300
487/487 [==============================] - 0s 68us/sample - loss: 0.0966 - accuracy: 0.0000e+00 - recall_6: 0.7642 - precision_6: 0.7460 - val_loss: 0.0983 - val_accuracy: 0.0000e+00 - val_recall_6: 0.8333 - val_precision_6: 0.7857
Epoch 195/300
487/487 [==============================] - 0s 77us/sample - loss: 0.0966 - accuracy: 0.0000e+00 - recall_6: 0.7642 - precision_6: 0.7460 - val_loss: 0.0982 - val_accuracy: 0.0000e+00 - val_recall_6: 0.8182 - val_precision_6: 0.7826
Epoch 196/300
487/487 [=======

487/487 [==============================] - 0s 61us/sample - loss: 0.0936 - accuracy: 0.0000e+00 - recall_6: 0.7886 - precision_6: 0.7519 - val_loss: 0.0983 - val_accuracy: 0.0000e+00 - val_recall_6: 0.8333 - val_precision_6: 0.7857
Epoch 226/300
487/487 [==============================] - 0s 83us/sample - loss: 0.0934 - accuracy: 0.0000e+00 - recall_6: 0.7886 - precision_6: 0.7519 - val_loss: 0.0981 - val_accuracy: 0.0000e+00 - val_recall_6: 0.8182 - val_precision_6: 0.7826
Epoch 227/300
487/487 [==============================] - 0s 70us/sample - loss: 0.0932 - accuracy: 0.0000e+00 - recall_6: 0.7886 - precision_6: 0.7519 - val_loss: 0.0977 - val_accuracy: 0.0000e+00 - val_recall_6: 0.8182 - val_precision_6: 0.7941
Epoch 228/300
487/487 [==============================] - 0s 98us/sample - loss: 0.0934 - accuracy: 0.0000e+00 - recall_6: 0.7642 - precision_6: 0.7520 - val_loss: 0.0976 - val_accuracy: 0.0000e+00 - val_recall_6: 0.8182 - val_precision_6: 0.8060
Epoch 229/300
487/487 [=======

487/487 [==============================] - 0s 66us/sample - loss: 0.0905 - accuracy: 0.0000e+00 - recall_6: 0.8130 - precision_6: 0.7576 - val_loss: 0.0987 - val_accuracy: 0.0000e+00 - val_recall_6: 0.8182 - val_precision_6: 0.7941
Epoch 259/300
487/487 [==============================] - 0s 72us/sample - loss: 0.0906 - accuracy: 0.0000e+00 - recall_6: 0.8130 - precision_6: 0.7519 - val_loss: 0.0989 - val_accuracy: 0.0000e+00 - val_recall_6: 0.8182 - val_precision_6: 0.7714
Epoch 260/300
487/487 [==============================] - 0s 144us/sample - loss: 0.0907 - accuracy: 0.0000e+00 - recall_6: 0.8211 - precision_6: 0.7537 - val_loss: 0.0990 - val_accuracy: 0.0000e+00 - val_recall_6: 0.8182 - val_precision_6: 0.7714
Epoch 261/300
487/487 [==============================] - 0s 67us/sample - loss: 0.0906 - accuracy: 0.0000e+00 - recall_6: 0.8130 - precision_6: 0.7519 - val_loss: 0.0989 - val_accuracy: 0.0000e+00 - val_recall_6: 0.8182 - val_precision_6: 0.7941
Epoch 262/300
487/487 [======

487/487 [==============================] - 0s 87us/sample - loss: 0.0880 - accuracy: 0.0000e+00 - recall_6: 0.8211 - precision_6: 0.7537 - val_loss: 0.1010 - val_accuracy: 0.0000e+00 - val_recall_6: 0.7879 - val_precision_6: 0.7879
Epoch 292/300
487/487 [==============================] - 0s 81us/sample - loss: 0.0881 - accuracy: 0.0000e+00 - recall_6: 0.8130 - precision_6: 0.7576 - val_loss: 0.1011 - val_accuracy: 0.0000e+00 - val_recall_6: 0.7879 - val_precision_6: 0.7879
Epoch 293/300
487/487 [==============================] - 0s 59us/sample - loss: 0.0880 - accuracy: 0.0000e+00 - recall_6: 0.8130 - precision_6: 0.7634 - val_loss: 0.1012 - val_accuracy: 0.0000e+00 - val_recall_6: 0.7879 - val_precision_6: 0.7879
Epoch 294/300
487/487 [==============================] - 0s 65us/sample - loss: 0.0876 - accuracy: 0.0000e+00 - recall_6: 0.8211 - precision_6: 0.7594 - val_loss: 0.1014 - val_accuracy: 0.0000e+00 - val_recall_6: 0.7879 - val_precision_6: 0.7761
Epoch 295/300
487/487 [=======

In [74]:
# confusion matrix
pred_Y = ANN_model.predict_classes(test_X);
confusion_matrix(test_Y, pred_Y)

array([[129,  14],
       [ 14,  52]])

In [75]:
# report evaluation metrics 
evaluated_metrics = ANN_model.evaluate(test_X, test_Y)
for i in range(len(metrics)):
    print(metrics_names[i] + ": %.2f" % evaluated_metrics[i])

209/209 [==============================] - 0s 104us/sample - loss: 0.1013 - accuracy: 0.0000e+00 - recall_6: 0.7879 - precision_6: 0.7879
accuracy: 0.10
recall: 0.00
precision: 0.79
